In [2]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil

# import plotly.io as pio
# pio.renderers.default = "browser"

In [2]:
def file_types(path):
    file_types=set()
    for filename in os.listdir(path):
        extension=os.path.splitext(filename)[1]
        if extension:
            file_types.add(extension)
    return file_types

<h1>Import to DataBase - sheep data

In [150]:
path='/mnt/c/Users/16162/Documents/Data/sheep_data/2023-04-20/'
filelist=[file for file in os.listdir(path) if '.csv' in file]

mylist=[]
for file in filelist:
    # print(file)
    mylist.append(file.split('-Frame')[0])
list(set(mylist))
for item in mylist:
    print(item)

REDO_SHEEP1_nocone_5degree_3mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
REDO_SHEEP1_nocone_5degree_3mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
REDO_SHEEP1_nocone_5degree_3mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
SHEEP1_cone_20degree_25mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
SHEEP1_cone_20degree_25mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
SHEEP1_cone_20degree_25mmrad_bothtags_spiked1_neck_prespike_785.092_100mW_20s_3frames_
SHEEP1_cone_20degree_25mmrad_bothtags_spiked2_SRDA_neck_postspike_785.092_100mW_20s_3frames_0minINC
SHEEP1_cone_20degree_25mmrad_bothtags_spiked2_SRDA_neck_postspike_785.092_100mW_20s_3frames_0minINC
SHEEP1_cone_20degree_25mmrad_bothtags_spiked2_SRDA_neck_postspike_785.092_100mW_20s_3frames_0minINC
SHEEP1_cone_20degree_25mmrad_bothtags_spiked2_SRDA_neck_postspike_785.092_100mW_20s_3frames_10minINC
SHEEP1_cone_20degree_25mmrad_bothtags_spiked2_SRDA_neck_postsp

In [167]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','sheep')
filelist=[file for file in os.listdir(path) if '.csv' in file]

mylist=[]

for file in filelist:
    df=pd.read_csv(path+file, header=None)
    x=df.iloc[:,0]
    x=[int(i) for i in x]
    x=np.array(x)
    xb=x.tobytes()
    
    y=df.iloc[:,1]
    y=np.array(y)
    yb=y.tobytes()

    splits=file.split('_')
    
    #Set default vaules
    last = splits[-1].split('-Frame')[0]
    time=last if 's' in last else 'NaN'
    spike='no'
    radius='0'
    degree='0'
    tag='0'
    frame='0'
    inc_time='NaN'
    power='NaN'
    

    if 'tag' in file and 'both_tag' not in file:
        tag='NBA'
    elif 'both_tag' in file:
        tag='both'
    elif 'tag' not in file:
        tag='no'

    #Set default vaules
    # last = splits[-1].split('-Frame')[0]
    # time=last if 's' in last else 'NaN'
    time='20s'
    spike='no'
    radius='0'
    degree='0'
    tag='0'
    frame='0'
    inc_time='NaN'
    power='NaN'
    surg='NaN'
    cone='no'

    if 'SHEEP1' in file:
        sheep='1'
    elif 'SHEEP2' in file:
        sheep='2'

    if 'cone' in file.lower() and 'nocone' not in file.lower():
        cone='yes'
    elif 'nocone' in file.lower():
        cone='no'

    if 'foil' in file.lower():
        foil='yes'
    elif 'foil' not in file.lower():
        foil='no'
        
    if 'tag' not in file:
        tag='0'
        print(file)
    elif 'tag' in file:
        tag='both'
    body='neck'
    

    for s in splits:
        # radius = s.split('mm')[0]+'mm' if 'mm' in s else '0mm'
        if 'srda' in s.lower():
            spike='yes'
        if 'mm' in s:     
            radius = s.split('mm')[0]+'mm'
        if 'degree' in s.lower():
            degree=s.split('deg')[0]
        if '-Frame-' in s:
            n1=s.split('-Frame-')[-1]
            frame=n1.split('.')[0]
        if 'min' in s:
            if '-Frame-' in s:
                t=s.split(' ')[0]
                inc_time=t.split('min')[0]+'min'
        if 'mw' in s.lower():
            power=s

        if 'surgery' in s:
            if 'pre' in s:
                surg='pre'
            elif 'post' in s:
                surg='post'
        if 'nm' in s.lower():
            wl=s
        

    # mylist.append(spike)
    
# print(list(set(mylist)))

    
    # db.insert_row(('raman_shift',xb),('original_intensity',yb),
    #               ('filename',file.split('.csv')[0]),('frame',frame),
    #               ('sheep',sheep),('date','2023-04-20'),
    #               ('foil',foil),('body_location',body),('surgery',surg),('cone',cone),
    #               ('spike',spike),('nanotag',tag),('degree',degree),('radius',radius),
    #               ('power',power),('wavelength',wl),
    #               ('incubation_time',inc_time), ('collect_time','20s')
    #               )
            
            

In [108]:
for file in filelist:
    if '20s' not in file.lower():
        print(file)

<h1>Import to DataBase</h1> 
CSV -> BD Table


In [35]:
path='/mnt/c/Users/16162/Documents/Data/2023-03-29/'
date='2023-03-29'
operator='WG'
# slice='1'

filelist=[file for file in os.listdir(path) if '.csv' in file and 'hexane' not in file]
# for file in filelist:
#     print(file)

for file in filelist:
    name=file.split('.csv')[0]
    if 'Frame' in file:
        frame=name.split('-')[2]
        name=name.split('-F')[0]
    else:
        frame=0
    
    s=name.split('_')
    general_loc=s[0].lower()
    loc_color=s[1].lower()
    slice=s[2][-1].lower()
    # slice='1'
    pow='10mW'
    time=s[-1].split(' ')[0]
    obj='40X'
    
    df=pd.read_csv(path+file,header=None)
    x=df.iloc[:,0]
    
    x=np.array(x)
    xb=x.tobytes()
    
    y=np.array(df.iloc[:,1])
    yb=y.tobytes()

    stmt=''
    # stmt=f'raman_shift:\t\t{xb}\n'
    # stmt+=f'original_intensity:\t{yb}\n'
    stmt+=f'filename:\t\t{name}\n'
    stmt+=f'frame:\t\t\t{int(frame)}\n'
    stmt+=f'date:\t\t\t{date}\n'
    stmt+=f'slice:\t\t\t{slice}\n'
    stmt+=f'power:\t\t\t{pow}\n'
    stmt+=f'time:\t\t\t{time}\n'
    stmt+=f'operator:\t\t{operator}\n'
    stmt+=f'general_loc:\t\t{general_loc}\n'
    stmt+=f'loc_color:\t\t{loc_color}\n'
    # print(stmt)

    # db.insert_row(('raman_shift',xb),
    #               ('original_intensity',yb),
    #               ('filename',name),
    #               ('frame',int(frame)),
    #               ('date',date),('slice',slice),
    #               ('power',pow),('time',time),
    #               ('general_loc',general_loc),
    #               ('loc_color',loc_color), ('operator_name',operator)
    #               )

db.columns('date')[1]


Values in date:
	2022-10-12
	2022-11-03
	2022-08-15
	2022-11-10
	2023-01-10
	2023-03-30
	2023-03-01
	2023-03-03
	2023-03-16
	2023-03-24
	2023-03-27
	2023-03-29


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,539,INTEGER,0,None,1
1,raman_shift,12,BLOB,0,None,0
2,original_intensity,446,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,222,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0


<h1>Edit values in database

In [57]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
db.columns('general_loc')[1]
# db.update('general_loc','cerebral cortex', 'cortex')


Values in general_loc:
	blank
	center
	slice2blank
	slice2wellplatediffplateposition
	slice2wellplate
	slice2
	slice3coverslip
	subniagraish
	cortex
	midbrain
	substantia nigra
	vta
	brainstem
	tissue2
	tissue3
	532nm
	ventrical
	low


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,539,INTEGER,0,None,1
1,raman_shift,12,BLOB,0,None,0
2,original_intensity,446,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,222,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0
